In [1]:
import pandas as pd
import codecs
import numpy as np
import math
import csv
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import math

In [2]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])

dat = pd.DataFrame()
dat['Timestamp'] = np.array(dat_source['Time-Date stamp'].iloc[1:])
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)

wind_df = wind_df.reset_index(drop=True)

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])

In [42]:
data_path = 'C:/Users/smartgrid_AI/Documents/kms/windpower_ramprate/Combined_training_result_df.csv'
data_source = pd.read_csv(data_path)
dat_source = pd.DataFrame(data_source)
dat_source = dat_source.iloc[:,1:]
dat_source['Generation Error'] = np.array(dat_source[['Real Generation']])-np.array(dat_source[['Predicted Generation']])
#dat_source['Ramp Error'] = np.array(dat_source[['Real Ramp']])-np.array(dat_source[['Predicted Ramp']])

dat_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error
0,0.599368,0.593079,-1.165505,-1.794430,-0.006289
1,0.566768,0.564236,-3.260029,-2.884298,-0.002532
2,0.549180,0.550182,-1.758769,-1.405453,0.001001
3,0.549626,0.541778,0.044534,-0.840336,-0.007847
4,0.523511,0.512642,-2.611463,-2.913655,-0.010869
...,...,...,...,...,...
749107,0.370860,0.363335,-1.513485,-2.502920,-0.007525
749108,0.367847,0.354841,-0.301263,-0.849347,-0.013006
749109,0.314876,0.301810,-5.297110,-5.303111,-0.013066
749110,0.249479,0.241135,-6.539671,-6.067523,-0.008344


In [43]:
dat_source = dat_source.reset_index(drop=True)

dat_source[['Real Ramp']] = std_scaler_ramp.transform(dat_source[['Real Ramp']])
dat_source[['Predicted Ramp']] = std_scaler_ramp.transform(dat_source[['Predicted Ramp']])
dat_source['Ramp Error'] = np.array(dat_source[['Predicted Ramp']])-np.array(dat_source[['Real Ramp']])

dat_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error,Ramp Error
0,0.599368,0.593079,0.477633,0.458694,-0.006289,0.018939
1,0.566768,0.564236,0.414561,0.425875,-0.002532,-0.011314
2,0.549180,0.550182,0.459768,0.470408,0.001001,-0.010639
3,0.549626,0.541778,0.514071,0.487425,-0.007847,0.026646
4,0.523511,0.512642,0.434091,0.424991,-0.010869,0.009100
...,...,...,...,...,...,...
749107,0.370860,0.363335,0.467155,0.437360,-0.007525,0.029795
749108,0.367847,0.354841,0.503658,0.487154,-0.013006,0.016504
749109,0.314876,0.301810,0.353218,0.353038,-0.013066,0.000181
749110,0.249479,0.241135,0.315801,0.330019,-0.008344,-0.014218


In [44]:
pred = np.array(dat_source[['Predicted Generation']])
rpred = np.array(dat_source[['Predicted Ramp']])
distance = np.sqrt(np.square(pred-np.mean(pred))+np.square(rpred-np.mean(rpred)))
dat_source['distance'] = distance

times = np.zeros(dat_source.shape[0], dtype=int)
for i in range(times.shape[0]):
    times[i] = int(i%24)

dat_source['time'] = times

dat_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error,Ramp Error,distance,time
0,0.599368,0.593079,0.477633,0.458694,-0.006289,0.018939,0.257894,0
1,0.566768,0.564236,0.414561,0.425875,-0.002532,-0.011314,0.243431,1
2,0.549180,0.550182,0.459768,0.470408,0.001001,-0.010639,0.211976,2
3,0.549626,0.541778,0.514071,0.487425,-0.007847,0.026646,0.205822,3
4,0.523511,0.512642,0.434091,0.424991,-0.010869,0.009100,0.195973,4
...,...,...,...,...,...,...,...,...
749107,0.370860,0.363335,0.467155,0.437360,-0.007525,0.029795,0.052610,19
749108,0.367847,0.354841,0.503658,0.487154,-0.013006,0.016504,0.025535,20
749109,0.314876,0.301810,0.353218,0.353038,-0.013066,0.000181,0.161672,21
749110,0.249479,0.241135,0.315801,0.330019,-0.008344,-0.014218,0.217951,22


In [45]:
data_path = 'C:/Users/smartgrid_AI/Documents/kms/windpower_ramprate/Combined_result_df.csv'
test_source = pd.read_csv(data_path)
test_source = pd.DataFrame(test_source)
test_source = test_source.iloc[:,1:]

test_source['Generation Error'] = np.array(test_source[['Real Generation']])-np.array(test_source[['Predicted Generation']])
test_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error
0,0.361006,0.352665,2.904399,2.070283,-0.008341
1,0.394062,0.387860,3.305594,3.519482,-0.006202
2,0.433283,0.427620,3.922059,3.976006,-0.005663
3,0.453140,0.455675,1.985712,2.805500,0.002535
4,0.411559,0.425549,-4.158111,-3.012528,0.013991
...,...,...,...,...,...
79603,0.172693,0.224535,1.495784,0.906008,0.051842
79604,0.191929,0.237918,1.923602,1.338303,0.045989
79605,0.204588,0.260646,1.265881,2.272786,0.056058
79606,0.199369,0.283270,-0.521911,2.262432,0.083901


In [46]:
test_source = test_source.reset_index(drop=True)

test_source[['Real Ramp']] = std_scaler_ramp.transform(test_source[['Real Ramp']])
test_source[['Predicted Ramp']] = std_scaler_ramp.transform(test_source[['Predicted Ramp']])
test_source['Ramp Error'] = np.array(test_source[['Predicted Ramp']])-np.array(test_source[['Real Ramp']])

test_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error,Ramp Error
0,0.361006,0.352665,0.600190,0.575072,-0.008341,0.025118
1,0.394062,0.387860,0.612271,0.618712,-0.006202,-0.006441
2,0.433283,0.427620,0.630835,0.632459,-0.005663,-0.001624
3,0.453140,0.455675,0.572526,0.597212,0.002535,-0.024686
4,0.411559,0.425549,0.387517,0.422014,0.013991,-0.034497
...,...,...,...,...,...,...
79603,0.172693,0.224535,0.557772,0.540013,0.051842,0.017760
79604,0.191929,0.237918,0.570655,0.553030,0.045989,0.017625
79605,0.204588,0.260646,0.550849,0.581170,0.056058,-0.030321
79606,0.199369,0.283270,0.497014,0.580858,0.083901,-0.083845


In [47]:
pred = np.array(test_source[['Predicted Generation']])
rpred = np.array(test_source[['Predicted Ramp']])
distance = np.sqrt(np.square(pred-np.mean(pred))+np.square(rpred-np.mean(rpred)))
test_source['distance'] = distance

times = np.zeros(test_source.shape[0], dtype=int)
for i in range(times.shape[0]):
    times[i] = int(i%24)

test_source['time'] = times


test_source

,Predicted Generation,Real Generation,Predicted Ramp,Real Ramp,Generation Error,Ramp Error,distance,time
0,0.361006,0.352665,0.600190,0.575072,-0.008341,0.025118,0.128934,0
1,0.394062,0.387860,0.612271,0.618712,-0.006202,-0.006441,0.161532,1
2,0.433283,0.427620,0.630835,0.632459,-0.005663,-0.001624,0.203846,2
3,0.453140,0.455675,0.572526,0.597212,0.002535,-0.024686,0.193989,3
4,0.411559,0.425549,0.387517,0.422014,0.013991,-0.034497,0.186669,4
...,...,...,...,...,...,...,...,...
79603,0.172693,0.224535,0.557772,0.540013,0.051842,0.017760,0.108697,19
79604,0.191929,0.237918,0.570655,0.553030,0.045989,0.017625,0.099347,20
79605,0.204588,0.260646,0.550849,0.581170,0.056058,-0.030321,0.077512,21
79606,0.199369,0.283270,0.497014,0.580858,0.083901,-0.083845,0.071487,22


In [48]:
from scipy.optimize import minimize_scalar
from scipy.stats import laplace, norm
import math

from scipy.stats import rv_continuous
import scipy

def dist_num(a, b, max_dist):
    x = np.sqrt(np.square(a-np.mean(pred))+np.square(b-np.mean(rpred)))
    if x<max_dist/6:
        num = 0
    elif x<max_dist/6*2:
        num = 1
    elif x<max_dist/6*3:
        num = 2
    elif x<max_dist/6*4:
        num = 3
    elif x<max_dist/6*5:
        num = 4
    else:
        num = 5
    return num

def dist_group(dat, dnum, max_dist):
    if dnum==0:
        X = dat.loc[dat.distance<max_dist/6, :]
    elif dnum==1:
        X = dat.loc[(max_dist/6<dat.distance) & (dat.distance<max_dist*2/6), :]
    elif dnum==2:
        X = dat.loc[(max_dist*2/6<dat.distance) & (dat.distance<max_dist*3/6), :]
    elif dnum==3:
        X = dat.loc[(max_dist*3/6<dat.distance) & (dat.distance<max_dist*4/6), :]
    elif dnum==4:
        X = dat.loc[(max_dist*4/6<dat.distance) & (dat.distance<max_dist*5/6), :]
    else:
        X = dat.loc[max_dist*5/6<dat.distance, :]
    return X

def pred_group_num(x, dat):
    vmax, vmin = max(dat['Predicted Generation']), min(dat['Predicted Generation'])
    
    if x<(vmin+(vmax-vmin)/5*1):
        num = 0
    elif x<(vmin+(vmax-vmin)/5*2):
        num = 1
    elif x<(vmin+(vmax-vmin)/5*3):
        num =2
    elif x<(vmin+(vmax-vmin)/5*4):
        num = 3
    else:
        num = 4
    #elif x<(vmin+(vmax-vmin)/6*5):
    #    num = 4
    #else:
    #    num = 5
    return num

def ramppred_group_num(x, dat):
    vmax, vmin = max(dat['Predicted Ramp']), min(dat['Predicted Ramp'])
    
    if x<(vmin+(vmax-vmin)/5*1):
        num = 0
    elif x<(vmin+(vmax-vmin)/5*2):
        num = 1
    elif x<(vmin+(vmax-vmin)/5*3):
        num = 2
    elif x<(vmin+(vmax-vmin)/5*4):
        num = 3
    else:
        num = 4
    return num

def pred_group(dat, gnum):
    vmax, vmin = max(dat['Predicted Generation']), min(dat['Predicted Generation'])
    
    if gnum==0:
        X = dat.loc[dat['Predicted Generation']<vmin+(vmax-vmin)/5*1, :]
    elif gnum==1:
        X = dat.loc[(vmin+(vmax-vmin)/5*1<dat['Predicted Generation']) & (dat['Predicted Generation']<vmin+(vmax-vmin)/5*2), :]
    elif gnum==2:
        X = dat.loc[(vmin+(vmax-vmin)/5*2<dat['Predicted Generation']) & (dat['Predicted Generation']<vmin+(vmax-vmin)/5*3), :]
    elif gnum==3:
        X = dat.loc[(vmin+(vmax-vmin)/5*3<dat['Predicted Generation']) & (dat['Predicted Generation']<vmin+(vmax-vmin)/5*4), :]
    else:
        X = dat.loc[vmin+(vmax-vmin)/5*4<dat['Predicted Generation'], :]
        
    #elif gnum==4:
    #    X = dat.loc[(vmin+(vmax-vmin)/6*4<dat.Predicted_Value) & (dat.Predicted_Value<vmin+(vmax-vmin)/6*5), :]
    #else:
    #    X = dat.loc[vmin+(vmax-vmin)/6*5<dat.Predicted_Value, :]
    return X

def ramppred_group(dat, gnum):
    vmax, vmin = max(dat['Predicted Ramp']), min(dat['Predicted Ramp'])
    
    if gnum==0:
        X = dat.loc[dat['Predicted Ramp']<vmin+(vmax-vmin)/5*1, :]
    elif gnum==1:
        X = dat.loc[(vmin+(vmax-vmin)/5*1<dat['Predicted Ramp']) & (dat['Predicted Ramp']<vmin+(vmax-vmin)/5*2), :]
    elif gnum==2:
        X = dat.loc[(vmin+(vmax-vmin)/5*2<dat['Predicted Ramp']) & (dat['Predicted Ramp']<vmin+(vmax-vmin)/5*3), :]
    elif gnum==3:
        X = dat.loc[(vmin+(vmax-vmin)/5*3<dat['Predicted Ramp']) & (dat['Predicted Ramp']<vmin+(vmax-vmin)/5*4), :]
    else:
        X = dat.loc[vmin+(vmax-vmin)/5*4<dat['Predicted Ramp'], :]
    return X

def compute_bound(lrb0, lrb1, lrb2, lrmu0, lrmu1, lrmu2):
    if (lrb1==0 or np.isnan(lrb1)) and (lrb2==0 or np.isnan(lrb2)):
        rf = 1/lrb0*np.exp(-(k2-lrmu0)/lrb0)
    else:
        if lrb1==0 or np.isnan(lrb1):
            rf = 1/lrb2*np.exp(-(k2-lrmu2)/lrb2)
        elif lrb2==0 or np.isnan(lrb2):
            rf = 1/lrb1*np.exp(-(k2-lrmu1)/lrb1)
        else:
            rK = lrb0/(2*lrb1*lrb2)
            ralpha = -1/lrb1-1/lrb2+1/lrb0
            rbeta = lrmu1/lrb1+lrmu2/lrb2-lrmu0/lrb0
            rf = rK/ralpha*np.exp(ralpha*k2+rbeta)
    return rf

def extract_groups(i, d_source):
    x = test_source.loc[i,'Predicted Generation']
    y = test_source.loc[i,'Predicted Ramp']    
    
    #maxdist = max(d_source.distance)
    #dnum = dist_num(x, y, maxdist)
    #inter_X = dist_group(d_source, dnum, maxdist)
    #N = inter_X.shape[0]

    gnum = pred_group_num(x, d_source)
    rnum = ramppred_group_num(y, d_source)
    p_group = pred_group(d_source, gnum)
    rp_group = ramppred_group(d_source, rnum)
    
    return p_group, rp_group

### Proposed Method

In [49]:
p1, p2, p3 = 0.9975, 0.995, 0.975

In [50]:
ubs = np.zeros((test_source.shape[0], 3))
lbs = np.zeros((test_source.shape[0], 3))

for i in range(test_source.shape[0]):
#for i in range(10):
    clock = int(i%24)
    d_source = dat_source.loc[dat_source.time==clock,:]
    lrmu0, lrb0 = laplace.fit(d_source['Ramp Error'])
    
    p_group, rp_group = extract_groups(i, d_source)

    ### Ramp Boundary Calculation ###
    lrmu1, lrb1 = laplace.fit(p_group['Ramp Error'])
    lrmu2, lrb2 = laplace.fit(rp_group['Ramp Error'])

    K = lrb0/(2*lrb1*lrb2)
    beta = -1/lrb1-1/lrb2+1/lrb0
    gamma = lrmu1/lrb1+lrmu2/lrb2-lrmu0/lrb0
    
    alpha995 = 0.005
    alpha99 = 0.01
    alpha95 = 0.05
    
    x995 = (np.log(-alpha995*beta/(2*K))-gamma)/beta
    x99 = (np.log(-alpha99*beta/(2*K))-gamma)/beta
    x95 = (np.log(-alpha95*beta/(2*K))-gamma)/beta
    
    y995 = (-np.log(alpha995*beta/(2*K))-gamma)/beta
    y99 = (-np.log(alpha99*beta/(2*K))-gamma)/beta
    y95 = (-np.log(alpha95*beta/(2*K))-gamma)/beta
    
    ubs[i,:] = np.array((x995, x99, x95))
    lbs[i,:] = np.array((y995, y99, y95))
    
    narr = pd.DataFrame(np.repeat(np.array(test_source.loc[i]).reshape(1,-1), 10, axis=0), columns=dat_source.columns)
    dat_source = pd.concat([dat_source, narr], axis=0)
    dat_source = dat_source.reset_index(drop=True)
    dat_source['time'] = np.array(dat_source['time'], dtype=int)

    #dat_source = pd.concat([dat_source, test_source.loc[i]], axis=0)
    
    if i%10000 == 0:
        print(str(i)+"th confidence interval is calculated.")

0th confidence interval is calculated.
10000th confidence interval is calculated.
20000th confidence interval is calculated.
30000th confidence interval is calculated.
40000th confidence interval is calculated.
50000th confidence interval is calculated.
60000th confidence interval is calculated.
70000th confidence interval is calculated.


In [51]:
np.savetxt('ramp_bounds_ub.txt',ubs)
np.savetxt('ramp_bounds_lb.txt',lbs)

### Gaussian Modeling 

In [11]:
ramp_bounds = np.zeros((test_source.shape[0], 3))

for i in range(test_source.shape[0]):
#for i in range(10):
    clock = int(i%24)
    d_source = dat_source.loc[dat_source.time==clock,:]
    mu, sig = norm.fit(d_source['Ramp Error'])
    
    ramp_bounds[i,0] = 2.807*sig
    ramp_bounds[i,1] = 2.58*sig
    ramp_bounds[i,2] = 1.96*sig
    
    narr = pd.DataFrame(np.repeat(np.array(test_source.loc[i]).reshape(1,-1), 10, axis=0), columns=dat_source.columns)
    dat_source = pd.concat([dat_source, narr], axis=0)
    dat_source = dat_source.reset_index(drop=True)
    dat_source['time'] = np.array(dat_source['time'], dtype=int)

    #dat_source = pd.concat([dat_source, test_source.loc[i]], axis=0)
    
    if i%10000 == 0:
        print(str(i)+"th confidence interval is calculated.")

0th confidence interval is calculated.
10000th confidence interval is calculated.
20000th confidence interval is calculated.
30000th confidence interval is calculated.
40000th confidence interval is calculated.
50000th confidence interval is calculated.
60000th confidence interval is calculated.
70000th confidence interval is calculated.


In [27]:
np.savetxt('ramp_bounds_gaussian.txt',ramp_bounds)

### Bootstrap Method

In [28]:
import random

ramp_bounds3 = np.zeros((test_source.shape[0], 3))
tr_err = np.array(dat_source['Ramp Error'])
te_err = np.array(test_source['Ramp Error'])
n_sam = 1000
n_cal = 100

sam_arr = np.zeros((n_sam, int(n_cal/2)))
sam_mean = np.zeros(test_source.shape[0])
sam_std = np.zeros(test_source.shape[0])

for i in range(test_source.shape[0]):
#for i in range(1):    
    for j in range(n_sam):
        k = np.random.randint(tr_err.shape[0], size=n_cal)
        sam = abs(tr_err[k].reshape(-1))
        ind = np.argpartition(sam, -int(n_cal/2))[:int(n_cal/2)]
        sam = sam[ind.reshape(-1)]
        sam_arr[j,:] = sam
        
    sam_mean[i] = np.mean(sam_arr)
    sam_std[i] = np.std(sam_arr)
    
    ramp_bounds3[i,0] = 2.807*sam_std[i]
    ramp_bounds3[i,1] = 2.58*sam_std[i]
    ramp_bounds3[i,2] = 1.96*sam_std[i]
    
    narr = np.repeat(te_err[i].reshape(-1), 10, axis=0)
    tr_err = np.concatenate((tr_err.reshape(-1), narr.reshape(-1)), axis=0)
    
np.savetxt('ramp_bounds_bootstrap.txt',ramp_bounds)

## Probabilistic Forecasting Evaluation

In [52]:
p_ram = std_scaler_ramp.inverse_transform(np.array(test_source['Predicted Ramp']).reshape(-1,1))
p_ram = p_ram.reshape(-1)
p_ram

array([ 2.90439913,  3.30559398,  3.92205947, ...,  1.26588075,
       -0.52191148,  2.45989713])

In [59]:
## For the Gaussian modeling-based method or Bootstrap method, change "ramp_bounds" according to the results
#u995 = np.array(test_source['Predicted Ramp']) + ramp_bounds2[:,0]
#l995 = np.array(test_source['Predicted Ramp']) - ramp_bounds2[:,0]
#u99 = np.array(test_source['Predicted Ramp']) + ramp_bounds2[:,1]
#l99 = np.array(test_source['Predicted Ramp']) - ramp_bounds2[:,1]
#u95 = np.array(test_source['Predicted Ramp']) + ramp_bounds2[:,2]
#l95 = np.array(test_source['Predicted Ramp']) - ramp_bounds2[:,2]

## This code is only available for the proposed method
u995 = np.array(test_source['Predicted Ramp']) + ubs[:,0]
l995 = np.array(test_source['Predicted Ramp']) - ubs[:,0]
u99 = np.array(test_source['Predicted Ramp']) + ubs[:,1]
l99 = np.array(test_source['Predicted Ramp']) - ubs[:,1]
u95 = np.array(test_source['Predicted Ramp']) + ubs[:,2]
l95 = np.array(test_source['Predicted Ramp']) - ubs[:,2]


u995 = std_scaler_ramp.inverse_transform(u995.reshape(-1,1))
u99 = std_scaler_ramp.inverse_transform(u99.reshape(-1,1))
u95 = std_scaler_ramp.inverse_transform(u95.reshape(-1,1))
l995 = std_scaler_ramp.inverse_transform(l995.reshape(-1,1))
l99 = std_scaler_ramp.inverse_transform(l99.reshape(-1,1))
l95 = std_scaler_ramp.inverse_transform(l95.reshape(-1,1))

In [60]:
def PICP(y, lb, ub):
    val = (y<ub) & (y>lb)
    return np.sum(val)/(y.shape[0])

def PINAW(y, lb, ub):
    R = max(y)-min(y)
    n = y.shape[0]
    return sum(ub-lb)/n/R
    
def CWC(y, lb, ub, mu, eta = 50):
    pin = PINAW(y, lb, ub)
    pic = PICP(y, lb, ub)
    
    if pic>mu or pic==mu:
        gam = 0
    else:
        gam = 1
    return pin*(1+gam*np.exp(-eta*(pic-mu)))

def IS(y, lb, ub, p):
    alpha = 1-p
    num = y.shape[0]
    Sscore = 0
    for i in range(num):
        delta = (ub[i]-lb[i])
        if y[i]<lb[i]:
            score = -2*alpha*delta-4*(lb[i]-y[i])
        elif y[i]>ub[i]:
            score = -2*alpha*delta-4*(y[i]-ub[i])
        else:
            score = -2*alpha*delta
        Sscore += score
    return Sscore/num

In [61]:
r_ram = std_scaler_ramp.inverse_transform(np.array(test_source['Real Ramp']).reshape(-1,1))
r_ram = r_ram.reshape(-1)

In [62]:
u995 = u995.reshape(-1)
l995 = l995.reshape(-1)
u99 = u99.reshape(-1)
l99 = l99.reshape(-1)
u95 = u95.reshape(-1)
l95 = l95.reshape(-1)

In [64]:
print("99.5% Confidence Interval Result on Ramp Forecasting Error: "+str(PICP(r_ram,l995, u995))+', '+ str(PINAW(r_ram,l995, u995))+', '+ str(CWC(r_ram, l995, u995,0.99))+', '+ str(IS(r_ram, l995, u995, 0.995)))
print("99% Confidence Interval Result on Ramp Forecasting Error: "+str(PICP(r_ram,l99, u99))+', '+ str(PINAW(r_ram,l99, u99))+', '+ str(CWC(r_ram, l99, u99,0.95))+', '+ str(IS(r_ram, l99, u99, 0.99)))
print("95% Confidence Interval Result on Ramp Forecasting Error: "+str(PICP(r_ram,l95, u95))+', '+ str(PINAW(r_ram,l95, u95))+', '+ str(CWC(r_ram, l95, u95,0.90))+', '+ str(IS(r_ram, l95, u95, 0.95)))

99.5% Confidence Interval Result on Ramp Forecasting Error: 0.9763591598834288, 0.20547162491781185, 0.6118765243871058, -0.110016391697751
99% Confidence Interval Result on Ramp Forecasting Error: 0.9562732388704652, 0.17824610907733915, 0.17824610907733915, -0.1937615404649787
95% Confidence Interval Result on Ramp Forecasting Error: 0.8313360466284796, 0.11503041894954757, 3.6781590775001027, -0.6989760390184893
